<a href="https://colab.research.google.com/github/alexeybondarev/Deep_learning_exercises/blob/main/DL2__homework_classification_Bondarev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [ ]:
#! pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
! pip install torchtext==0.8.1

     |████████████████████████████████| 7.0MB 7.7MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchtext 0.9.0
    Uninstalling torchtext-0.9.0:
      Successfully uninstalled torchtext-0.9.0


In [ ]:
import pandas as pd
import numpy as np
import torch

from torchtext import datasets

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [ ]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [ ]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 55.1MB/s]
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [ ]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [ ]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        '''
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        '''
        self.rnn = nn.GRU(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)

        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        # cell arg for LSTM, remove for GRU
        packed_output, (hidden) = self.rnn(packed_embedded)
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
            
        return self.fc(hidden)

Поиграйтесь с гиперпараметрами

In [ ]:
vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 100
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.4
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=3

In [ ]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [ ]:
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

max_epochs = 20

In [ ]:
from sklearn.metrics import f1_score, precision_recall_fscore_support

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths.cpu()).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    epoch_loss /= len(train_iter)
    epoch_acc /= len(train_iter)
    val_loss = 0.0
    val_acc = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        # YOUR CODE GOES HERE
        text, text_lengths = batch.text
        predictions = model(text, text_lengths.cpu()).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        val_loss += loss.item()
        val_acc += acc.item()

    val_loss /= len(valid_iter)
    val_acc /= len(valid_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}, Training accuracy: {}, Validation accuracy: {}'.format(epoch, train_loss, val_loss, epoch_acc, val_acc))
model.load_state_dict(best_model)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 1, Training Loss: 0.0, Validation Loss: 0.5308252070414818, Training accuracy: 0.6369835116567403, Validation accuracy: 0.7321680794327946


Epoch: 2, Training Loss: 0.0, Validation Loss: 0.4214790736214589, Training accuracy: 0.8002069213964643, Validation accuracy: 0.8059675142926684


Epoch: 3, Training Loss: 0.0, Validation Loss: 0.38036659738775025, Training accuracy: 0.8874315695606009, Validation accuracy: 0.8320091811277098


Epoch: 4, Training Loss: 0.0, Validation Loss: 0.4287775278849117, Training accuracy: 0.9469336549731067, Validation accuracy: 0.8437941387548284


Epoch: 5, Training Loss: 0.0, Validation Loss: 0.4990402331923024, Training accuracy: 0.9761633211678832, Validation accuracy: 0.8483403956486006


<All keys matched successfully>

Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**:

In [ ]:
target_real = []
target_predicted = []
model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
for it, batch in pbar:
        # YOUR CODE GOES HERE
    text, text_lengths = batch.text
    predictions = model(text, text_lengths.cpu()).squeeze(1)
    loss = criterion(predictions, batch.label)
    target_real.extend(batch.label.cpu().numpy())
    target_predicted.extend(torch.round(torch.sigmoid(predictions)).cpu().detach().numpy())
f1score = f1_score(target_real, target_predicted, average="binary")
print ('f1score = ', f1score)    


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


f1score =  0.8434942636751634


## RNN сеть с GRU ячейками дает 0,843 f1score на тестовой выборке

## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [ ]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes fo

In [ ]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [ ]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dim,
        n_filters, 
        filter_sizes,
        dropout=0.4,
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[0], embedding_dim))
        
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[1], embedding_dim))
        
        self.conv_2 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, 1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text):
        
        embedded = self.embedding(text)
        
        embedded = embedded.unsqueeze(1)  # may be reshape here
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))  # may be reshape here
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))  # may be reshape here
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
            
        return self.fc(cat)

In [ ]:
filter_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
n_filters=64
dropout = 0.4
dim = 100

model = CNN(vocab_size=vocab_size, embedding_dim=dim, n_filters=n_filters,
            filter_sizes=filter_sizes, dropout=dropout)

In [ ]:
model.to(device)

CNN(
  (embedding): Embedding(202268, 100)
  (conv_0): Conv2d(1, 64, kernel_size=(3, 100), stride=(1, 1))
  (conv_1): Conv2d(1, 64, kernel_size=(4, 100), stride=(1, 1))
  (conv_2): Conv2d(1, 64, kernel_size=(5, 100), stride=(1, 1))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [ ]:
max_epochs = 30

Обучите!

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    epoch_loss = 0.0
    epoch_acc = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    epoch_loss /= len(train_iter)
    epoch_acc /= len(train_iter)
    val_loss = 0.0
    val_acc = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        # YOUR CODE GOES HERE
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        val_loss += loss.item()
        val_acc += acc.item()

    val_loss /= len(val_iter)
    val_acc /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}, Training accuracy: {}, Validation accuracy: {}'.format(epoch, train_loss, val_loss, epoch_acc, val_acc))
model.load_state_dict(best_model)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 1, Training Loss: 0.0, Validation Loss: 0.562816733121872, Training accuracy: 0.583991193423306, Validation accuracy: 0.7336554268995921


Epoch: 2, Training Loss: 0.0, Validation Loss: 0.49291690687338513, Training accuracy: 0.714768129108596, Validation accuracy: 0.7658511519432067


Epoch: 3, Training Loss: 0.0, Validation Loss: 0.4428968737522761, Training accuracy: 0.7634034831158436, Validation accuracy: 0.7978275775909424


Epoch: 4, Training Loss: 0.0, Validation Loss: 0.41518678863843284, Training accuracy: 0.7980452636732672, Validation accuracy: 0.8102316339810689


...

Epoch: 8, Training Loss: 0.0, Validation Loss: 0.3407839486996333, Training accuracy: 0.8947035665929752, Validation accuracy: 0.8516858557860056


Epoch: 9, Training Loss: 0.0, Validation Loss: 0.3347028950850169, Training accuracy: 0.9116723859397164, Validation accuracy: 0.8557702859242757


Epoch: 10, Training Loss: 0.0, Validation Loss: 0.3394787559906642, Training accuracy: 0.9298734530045168, Validation accuracy: 0.8565378288427988


Epoch: 11, Training Loss: 0.0, Validation Loss: 0.3356086189548175, Training accuracy: 0.9468769835729669, Validation accuracy: 0.8572368423144022


<All keys matched successfully>

In [ ]:
target_real = []
target_predicted = []
model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
for it, batch in pbar:
        # YOUR CODE GOES HERE
    text = batch.text
    predictions = model(text).squeeze(1)
    loss = criterion(predictions, batch.label)
    target_real.extend(batch.label.cpu().numpy())
    target_predicted.extend(torch.round(torch.sigmoid(predictions)).cpu().detach().numpy())
f1score = f1_score(target_real, target_predicted, average="binary")
print ('f1score = ', f1score) 

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


f1score =  0.8547651006711409


Посчитайте f1-score вашего классификатора.

**Ответ**:

## CNN сеть дает 0,855 f1score на тестовой выборке

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [ ]:
!pip install -q captum

     |████████████████████████████████| 4.4MB 8.2MB/s 


In [ ]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [ ]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [ ]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 1.00 ) , delta:  tensor([3.7975e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([1.2920e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.24 ) , delta:  tensor([9.1148e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([9.2720e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.18 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.37 ) , delta:  tensor([3.6368e-05], device='cuda:0', dtype=torch.float64)


In [ ]:
interpret_sentence(model, 'That shit sucks', label=0)
interpret_sentence(model, 'It is such a profound experience', label=1)
interpret_sentence(model, 'I liked it a lot', label=1)

pred:  neg ( 0.00 ) , delta:  tensor([2.4604e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.13 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.82 ) , delta:  tensor([0.0003], device='cuda:0', dtype=torch.float64)


### Интерпертируемость мягко говоря не очень, такое ощущение, что почти все считает отрицательным

Попробуйте добавить свои примеры!

In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),pos,1.69,It was a fantastic performance ! pad
pos,neg (0.00),pos,1.39,Best film ever pad pad pad pad
pos,neg (0.24),pos,1.37,Such a great show! pad pad pad
neg,neg (0.00),pos,0.54,It was a horrible movie pad pad
neg,neg (0.18),pos,1.47,I've never watched something as bad pad
neg,neg (0.37),pos,1.74,It is a disgusting movie! pad pad
neg,neg (0.00),pos,0.70,That shit sucks pad pad pad pad
neg,neg (0.00),pos,0.70,That shit sucks pad pad pad pad
pos,neg (0.13),pos,1.85,It is such a profound experience pad
neg,neg (0.00),pos,0.70,That shit sucks pad pad pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),pos,1.69,It was a fantastic performance ! pad
pos,neg (0.00),pos,1.39,Best film ever pad pad pad pad
pos,neg (0.24),pos,1.37,Such a great show! pad pad pad
neg,neg (0.00),pos,0.54,It was a horrible movie pad pad
neg,neg (0.18),pos,1.47,I've never watched something as bad pad
neg,neg (0.37),pos,1.74,It is a disgusting movie! pad pad
neg,neg (0.00),pos,0.70,That shit sucks pad pad pad pad
neg,neg (0.00),pos,0.70,That shit sucks pad pad pad pad
pos,neg (0.13),pos,1.85,It is such a profound experience pad
neg,neg (0.00),pos,0.70,That shit sucks pad pad pad pad


## Эмбэдинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [ ]:
TEXT.build_vocab(trn, vectors="glove.6B.100d")# YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
LABEL.build_vocab(trn)

word_embeddings = TEXT.vocab.vectors









.vector_cache/glove.6B.zip: 0.00B [00:00, ?B/s]





.vector_cache/glove.6B.zip:   0%|          | 8.19k/862M [00:00<7:43:34, 31.0kB/s]





.vector_cache/glove.6B.zip:   0%|          | 221k/862M [00:00<5:26:34, 44.0kB/s] 





.vector_cache/glove.6B.zip:   0%|          | 2.53M/862M [00:00<3:48:10, 62.8kB/s]





.vector_cache/glove.6B.zip:   1%|          | 8.58M/862M [00:00<2:38:40, 89.7kB/s]





.vector_cache/glove.6B.zip:   2%|▏         | 15.0M/862M [00:00<1:50:18, 128kB/s] 





.vector_cache/glove.6B.zip:   3%|▎         | 21.8M/862M [00:00<1:16:39, 183kB/s]





.vector_cache/glove.6B.zip:   3%|▎         | 27.0M/862M [00:00<53:28, 260kB/s]  





.vector_cache/glove.6B.zip:   4%|▎         | 31.2M/862M [00:01<38:02, 364kB/s]





.vector_cache/glove.6B.zip:   4%|▍         | 35.4M/862M [00:02<27:16, 505kB/s]





.vector_cache/glove.6B.zip:   4%|▍         | 38.8M/862M [00:02<19:08, 717kB/s]





.vector_cache/glove.6B.zip:   5%|▍         | 39.5M/862M [00:03<17:02, 804kB/s]





In [ ]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.4
dim = 100

In [ ]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
model = CNN(vocab_size=vocab_size, embedding_dim=dim, n_filters=n_filters,
            filter_sizes=filter_sizes, dropout=dropout)

word_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

model.embedding.weight.data.copy_(word_embeddings)

assert prev_shape == model.embedding.weight.shape
model.to(device)

optimizer = torch.optim.Adam(model.parameters())

Вы знаете, что делать.

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    epoch_loss = 0.0
    epoch_acc = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    epoch_loss /= len(train_iter)
    epoch_acc /= len(train_iter)
    val_loss = 0.0
    val_acc = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        # YOUR CODE GOES HERE
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        val_loss += loss.item()
        val_acc += acc.item()

    val_loss /= len(val_iter)
    val_acc /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}, Training accuracy: {}, Validation accuracy: {}'.format(epoch, train_loss, val_loss, epoch_acc, val_acc))
model.load_state_dict(best_model)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 1, Training Loss: 0.0, Validation Loss: 0.42410344978173575, Training accuracy: 0.6964480524515584, Validation accuracy: 0.8115474243958791


Epoch: 2, Training Loss: 0.0, Validation Loss: 0.35621233185132345, Training accuracy: 0.8404846678685097, Validation accuracy: 0.8463130493958791


Epoch: 3, Training Loss: 0.0, Validation Loss: 0.31495866576830545, Training accuracy: 0.8989656062891883, Validation accuracy: 0.8658922692139943


Epoch: 4, Training Loss: 0.0, Validation Loss: 0.30843538294235867, Training accuracy: 0.9487464297426879, Validation accuracy: 0.8734923243522644


Epoch: 5, Training Loss: 0.0, Validation Loss: 0.31477098216613136, Training accuracy: 0.9783873175182481, Validation accuracy: 0.8732319076855978


Epoch: 6, Training Loss: 0.0, Validation Loss: 0.33450489391883215, Training accuracy: 0.9933280109489051, Validation accuracy: 0.8692297160625457


<All keys matched successfully>

Посчитайте f1-score вашего классификатора.

**Ответ**:

In [ ]:
target_real = []
target_predicted = []
model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
for it, batch in pbar:
        # YOUR CODE GOES HERE
    text = batch.text
    predictions = model(text).squeeze(1)
    loss = criterion(predictions, batch.label)
    target_real.extend(batch.label.cpu().numpy())
    target_predicted.extend(torch.round(torch.sigmoid(predictions)).cpu().detach().numpy())
f1score = f1_score(target_real, target_predicted, average="binary")
print ('f1score = ', f1score) 

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


f1score =  0.8623427384878466


## CNN сеть c преобученными эмебддингами дает 0,862 f1score на тестовой выборке, прирост незначительный и наверняка его можно повысить, например замораживая эмебддинги на первых этапах обучения

Проверим насколько все хорошо!

In [ ]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.96 ) , delta:  tensor([0.0005], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.03 ) , delta:  tensor([2.3454e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.33 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([9.1401e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.39 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([4.1815e-05], device='cuda:0', dtype=torch.float64)


In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,2.00,It was a fantastic performance ! pad
pos,neg (0.03),pos,0.27,Best film ever pad pad pad pad
pos,neg (0.33),pos,1.59,Such a great show! pad pad pad
neg,neg (0.01),pos,-0.29,It was a horrible movie pad pad
neg,neg (0.39),pos,1.83,I've never watched something as bad pad
neg,neg (0.01),pos,-0.26,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,2.00,It was a fantastic performance ! pad
pos,neg (0.03),pos,0.27,Best film ever pad pad pad pad
pos,neg (0.33),pos,1.59,Such a great show! pad pad pad
neg,neg (0.01),pos,-0.29,It was a horrible movie pad pad
neg,neg (0.39),pos,1.83,I've never watched something as bad pad
neg,neg (0.01),pos,-0.26,It is a disgusting movie! pad pad
